# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 7 2022 7:56AM,250119,19,MSP215505,"Methapharm, Inc.",Released
1,Nov 7 2022 7:56AM,250119,19,MSP215508,"Methapharm, Inc.",Released
2,Nov 7 2022 7:56AM,250119,19,MSP215523,"Methapharm, Inc.",Released
3,Nov 7 2022 7:56AM,250119,19,MSP215559,"Methapharm, Inc.",Released
4,Nov 7 2022 7:53AM,250118,19,ADV80007391,"AdvaGen Pharma, Ltd",Released
5,Nov 7 2022 7:53AM,250118,19,ADV80007392,"AdvaGen Pharma, Ltd",Released
6,Nov 7 2022 7:53AM,250118,19,ADV80007393,"AdvaGen Pharma, Ltd",Released
7,Nov 7 2022 7:53AM,250118,19,ADV80007396,"AdvaGen Pharma, Ltd",Released
8,Nov 7 2022 7:53AM,250118,19,ADV80007397,"AdvaGen Pharma, Ltd",Released
9,Nov 7 2022 7:53AM,250118,19,ADV80007402,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
0,250116,Executing,3
1,250116,Released,4
2,250117,Released,32
3,250118,Released,6
4,250119,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250116,3.0,4.0
250117,NaN,32.0
250118,NaN,6.0
250119,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250116,3.0,4.0
250117,0.0,32.0
250118,0.0,6.0
250119,0.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250116,3,4
250117,0,32
250118,0,6
250119,0,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250116,3,4
1,250117,0,32
2,250118,0,6
3,250119,0,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250116,3,4
1,250117,,32
2,250118,,6
3,250119,,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 7 2022 7:56AM,250119,19,"Methapharm, Inc."
4,Nov 7 2022 7:53AM,250118,19,"AdvaGen Pharma, Ltd"
10,Nov 7 2022 7:49AM,250117,19,"Innogenix, LLC"
42,Nov 7 2022 7:39AM,250116,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 7 2022 7:56AM,250119,19,"Methapharm, Inc.",,4
1,Nov 7 2022 7:53AM,250118,19,"AdvaGen Pharma, Ltd",,6
2,Nov 7 2022 7:49AM,250117,19,"Innogenix, LLC",,32
3,Nov 7 2022 7:39AM,250116,20,"Exact-Rx, Inc.",3,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 7:56AM,250119,19,"Methapharm, Inc.",4,
1,Nov 7 2022 7:53AM,250118,19,"AdvaGen Pharma, Ltd",6,
2,Nov 7 2022 7:49AM,250117,19,"Innogenix, LLC",32,
3,Nov 7 2022 7:39AM,250116,20,"Exact-Rx, Inc.",4,3


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 7:56AM,250119,19,"Methapharm, Inc.",4,
1,Nov 7 2022 7:53AM,250118,19,"AdvaGen Pharma, Ltd",6,
2,Nov 7 2022 7:49AM,250117,19,"Innogenix, LLC",32,
3,Nov 7 2022 7:39AM,250116,20,"Exact-Rx, Inc.",4,3


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 7:56AM,250119,19,"Methapharm, Inc.",4,NaN
1,Nov 7 2022 7:53AM,250118,19,"AdvaGen Pharma, Ltd",6,NaN
2,Nov 7 2022 7:49AM,250117,19,"Innogenix, LLC",32,NaN
3,Nov 7 2022 7:39AM,250116,20,"Exact-Rx, Inc.",4,3.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 7:56AM,250119,19,"Methapharm, Inc.",4,0.0
1,Nov 7 2022 7:53AM,250118,19,"AdvaGen Pharma, Ltd",6,0.0
2,Nov 7 2022 7:49AM,250117,19,"Innogenix, LLC",32,0.0
3,Nov 7 2022 7:39AM,250116,20,"Exact-Rx, Inc.",4,3.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
19,750354,42,0.0
20,250116,4,3.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,19,750354,42,0.0
1,20,250116,4,3.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,19,42,0.0
1,20,4,3.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,19,Released,42.0
1,20,Released,4.0
2,19,Executing,0.0
3,20,Executing,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,19,20
Status,,
Executing,0.0,3.0
Released,42.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,19,20
0,Executing,0.0,3.0
1,Released,42.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,19,20
0,Executing,0.0,3.0
1,Released,42.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()